In [ ]:
#globals().clear()

## Download GEFS data

The model used here is Global Ensemble Forecast System (GEFS): https://www.ncei.noaa.gov/products/weather-climate-models/global-ensemble-forecast

The GEFS model product could be any of the following: 
- 'atmos.5' - Half degree atmos PRIMARY fields (pgrb2ap5); ~83 most common variables.
- 'atmos.5b' - Half degree atmos SECONDARY fields (pgrb2bp5); ~500 least common variables
- 'atmos.25' - Quarter degree atmos PRIMARY fields (pgrb2sp25); ~35 most common variables
- 'wave' - Global wave products. - 'chem.5'
- 'chem.5' - Chemistry fields on 0.5 degree grid
- 'chem.25' - Chemistry fields on 0.25 degree grid

For our case, following this: https://www.nco.ncep.noaa.gov/pmb/products/gens/, we will need:
- From atmos.25, surface: "sp", "tp", "sdswrf", "sdlwrf"
- From atmos.25, heightAboveGround 2m : "t2m", "d2m", "r2", "tmax", "tmin", "u10", "v10", "tcc", "st", "soilw"
- From atmos.25, heightAboveGround 10m :"u10", "v10",  
- From atmos.25, atmosphere: "tcc"
- From atmos.25, depthBelowLandLayer at 0-0.1m: "st", "soilw"
- From atmos.5b, depthBelowLandLayer at 0.1-0.4, 0.4-1, 1-2m: "st", "soilw" 

How do the members work? For the atmos output, member 0 is "c00", which correspond to the control member, then members 1-30 or "p01"-"p30". Another option could be 'avg' or 'mean' for the ensemble mean, 'spr' for ensemble spread.

How does the lead time work? here is represented by fxx in the herbie function, correspond to the perid that is been forecasted, the resolution is every 3 hours for GEFS. For instance, if I need the forecast for the next coming 2 days starting at 30/01/2025, I need to set fxx from 0 to 48 every 3 hours, in python will be range(0, 49, 3)

In [2]:
import numpy as np
import xarray as xr
from datetime import datetime, timedelta
from herbie import Herbie
import os
import time

In [3]:
# Define parameters
#initial = datetime.now() - timedelta(days=2)
#finish = datetime.now() - timedelta(days=1)
#start_date = datetime(initial.year, initial.month, initial.day, 0, 0)
#end_date = datetime(finish.year, finish.month, finish.day, 18, 0)
start_date = datetime(2020, 3, 1, 0, 0) - timedelta(days=2)
end_date = datetime(2020, 3, 10, 0, 0) - timedelta(days=1)
member_list = ["c00"] + [f"p{str(i).zfill(2)}" for i in range(1, 31)]
lead_times = range(0, 169, 3)  # Lead times from 0 to 168, every 3 hours = 7 days
#variables_surface = ["sp", "tp", "sdswrf", "sdlwrf"]
variables_surface = ["tp"]
#variables_2m = ["t2m", "d2m", "r2", "tmax", "tmin"]
variables_2m = ["t2m"]
#variables_10m = ["u10", "v10"]
#variables_atm = ["tcc"]
variables_soil = ["st", "soilw"]
#variables_accumulated = ["tp", "sdswrf", "sdlwrf", "tcc"]  # Variables to reset at 00:00
variables_accumulated = ["tp"]  # Variables to reset at 00:00
save_dir = "data/"
#coordinates for Sau Reservoir
lat, lon = 41.97, 2.39

In [4]:
init_per_day = 1 #initially tested with 4 times per day (take a lot of time)
init_resolution = 24 #initially tested with 6 hours (take a lot of time)
init_dates = np.array([start_date + timedelta(hours=init_resolution*i) for i in range(((end_date - start_date).days + 1) * init_per_day)], dtype="datetime64[ns]")

In [5]:
# Initialize an empty xarray dataset
#init_dates = np.array([start_date + timedelta(hours=6*i) for i in range(((end_date - start_date).days + 1) * 4)], dtype="datetime64[ns]")
dataset = xr.Dataset(
    coords={
        "member": range(0,len(member_list)),
        "init": init_dates,
        "lead": range(0,len(lead_times)),
        "latitude": lat,
        "longitude": lon
    }
)

In [9]:
                H = Herbie(
                    current_date.strftime("%Y-%m-%d %H:%M"),
                    model="gefs",
                    product='atmos.5',
                    member=member,
                    fxx=lead
                )

✅ Found ┊ model=gefs ┊ product=atmos.5 ┊ 2020-Feb-28 00:00 UTC F00 ┊ GRIB2 @ aws ┊ IDX @ aws


In [6]:
# Iterate over initialization dates
current_date = start_date
c_init = 0
while current_date <= end_date:
    c_member = 0
    for member in member_list:
        c_lead = 0
        time.sleep(10)
        for lead in lead_times:
                print("date: " + str(current_date) + " member: " + member + " lead: " + str(lead))
            #try:
                # Initialize Herbie for the current date, member, and lead
                H = Herbie(
                    current_date.strftime("%Y-%m-%d %H:%M"),
                    model="gefs",
                    product='atmos.25',
                    member=member,
                    fxx=lead
                )
                
                # Download the data
                path = H.download(save_dir=save_dir)

                # Initialize a dictionary to hold all data for this combination
                data = {}

                # Surface variables
                ds = xr.open_dataset(path, engine="cfgrib", filter_by_keys={'typeOfLevel': 'surface'})
                ds_sel = ds.sel(latitude=lat, longitude=lon, method="nearest")
                for var in variables_surface:
                    #if var in variables_accumulated and current_date.strftime("%H:%M") == "00:00":
                    if var in variables_accumulated and lead == 0:
                        data[var] = np.array(0, dtype="float32")  # Set accumulated variables to 0
                    else:
                        data[var] = ds_sel[var].values

                # 2m height variables
                ds = xr.open_dataset(path, engine="cfgrib", filter_by_keys={'typeOfLevel': 'heightAboveGround', 'level': 2})
                ds_sel = ds.sel(latitude=lat, longitude=lon, method="nearest")
                for var in variables_2m:
                    data[var] = ds_sel[var].values

                # 10m height variables
                #ds = xr.open_dataset(path, engine="cfgrib", filter_by_keys={'typeOfLevel': 'heightAboveGround', 'level': 10})
                #ds_sel = ds.sel(latitude=lat, longitude=lon, method="nearest")
                #for var in variables_10m:
                #    data[var] = ds_sel[var].values

                # Atmosphere variables
                #for var in variables_atm:
                    #if var in variables_accumulated and current_date.strftime("%H:%M") == "00:00":
                #    if var in variables_accumulated and lead == 0:
                #        data[var] = np.array(0, dtype="float32")  # Set accumulated variables to 0
                #    else:
                #        ds = xr.open_dataset(path, engine="cfgrib", filter_by_keys={'typeOfLevel': 'atmosphere'})
                #        ds_sel = ds.sel(latitude=lat, longitude=lon, method="nearest")
                #        data[var] = ds_sel[var].values

                # Below-land variables (only for 0-10 cm for soil)
                ds = xr.open_dataset(path, engine="cfgrib", filter_by_keys={'typeOfLevel': 'depthBelowLandLayer'})
                ds_sel = ds.sel(latitude=lat, longitude=lon, method="nearest")
                for var in variables_soil:
                    data[var] = ds_sel[var].values

                #secondary variables (soil for different depths) at 5 degrees
                H = Herbie(
                    current_date.strftime("%Y-%m-%d %H:%M"),
                    model="gefs",
                    product="atmos.5b",
                    member=member,
                    fxx=lead
                )

                # Download the data
                path = H.download(save_dir=save_dir)

                # soil temperature at different depths
                ds = xr.open_dataset(path, engine="cfgrib", filter_by_keys={'paramId': 228139})  # st variable
                ds_sel = ds.sel(latitude=lat, longitude=lon, method="nearest")
                data["st_01"] = ds_sel.sel(depthBelowLandLayer=0.1)['st'].values
                data["st_04"] = ds_sel.sel(depthBelowLandLayer=0.4)['st'].values
                data["st_1"] = ds_sel.sel(depthBelowLandLayer=1)['st'].values

                # Volumetric Soil Moisture Content at different depths
                ds = xr.open_dataset(path, engine="cfgrib", filter_by_keys={'paramId': 260185})  # soilw variable
                ds_sel = ds.sel(latitude=lat, longitude=lon, method="nearest")
                data["soilw_01"] = ds_sel.sel(depthBelowLandLayer=0.1)['soilw'].values
                data["soilw_04"] = ds_sel.sel(depthBelowLandLayer=0.4)['soilw'].values
                data["soilw_1"] = ds_sel.sel(depthBelowLandLayer=1)['soilw'].values

                # Append data to the xarray dataset
                for var, values in data.items():
                    if var not in dataset:
                        dataset[var] = (("member", "init", "lead"), np.full((len(member_list), len(range((end_date - start_date).days + 1))*4, len(lead_times)), np.nan))
                    dataset[var][c_member, c_init, c_lead] = values

                os.remove(path)

                # Remove the file after processing
                #if os.path.exists(path):
                    #os.remove(path)

                c_lead += 1

            #except Exception as e:
            #    print(f"Error processing {current_date} {member} {lead}: {e}")

        c_member += 1
    c_init += 1

    # Increment the date
    current_date += timedelta(hours=init_resolution)

# Final dataset is now built
print(dataset)


date: 2020-02-28 00:00:00 member: c00 lead: 0


ValueError: product=atmos.25 not recognized. Must be one of dict_keys(['atmos.5', 'atmos.5b', 'atmos.25', 'wave', 'chem.5', 'chem.25'])

In [ ]:
# Save dataset
dataset.to_netcdf("gefs_"+ str(pd.to_datetime(init_dates[0]).date()))

In [322]:
#select a day to save
day_save = "2025-01-27"
ds_selected = dataset.sel(init=day_save)
ds_selected.to_netcdf("gefs_"+ day_save)